In [17]:
import os
import cv2
import numpy as np
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import time 


In [18]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = load_model('./trained_models/inception_sgd_classweights.h5')



In [19]:

# To capture video from webcam. 
cap = cv2.VideoCapture(0)
# To use a video file as input 
# cap = cv2.VideoCapture('filename.mp4')

while True:
    # Read the frame
    _, img = cap.read()
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect the faces
    faces = face_haar_cascade.detectMultiScale(gray)
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+w,x:x+h]
        roi_gray=cv2.resize(roi_gray,(75,75))
        roi_gray = np.true_divide(roi_gray,255)
        roi_gray_3dim = np.stack((roi_gray,)*3, axis=-1)
        roi_gray_3dim = roi_gray_3dim.reshape(-1, 75,75,3)
        #plt.imshow(roi_gray_3dim[0])
        #print(roi_gray_3dim.shape)
        predictions = model.predict(roi_gray_3dim)
        max_index = np.argmax(predictions[0])

        emotion_detection = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
        emotion_prediction = emotion_detection[max_index]

        cv2.putText(img, emotion_prediction, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX ,0.7, (0,255,0), 1, cv2.LINE_AA)
        #time.sleep(10000)
        
        
    # Display
    cv2.imshow('img', img)
    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
# Release the VideoCapture object
cap.release()
cv2.destroyAllWindows()
